# TP11 : Compter le nombre de configurations du morpion

Le but de ce TP est de compter le nombre de configurations possibles du jeu de morpion.

Rappel des règles du jeu:
* jeu de plateau $3\times 3$ à deux joueurs, respectivement X et O
* X commence et les joueurs jouent alternativement
* à son tour, chaque joueur inscrit sa marque (X ou O) dans une case libre
* un joueur gagne quand il a réussi à aligner 3 marques
* le jeu se termine quand il n'y a plus de case libre ou qu'un des joueurs a gagné 


Dans toute la suite, on représentera une grille par un 9-uplet dont chaque composante contient une valeur parmi `'X'`, `'O'` et `'?'` et qui correspond à une lecture ligne à ligne de la grille. (Le symbole `'?'` représente une case vide.)

**exemple**: la grille

```bash
-------------
| X | O | X |
-------------
|   | X | O |
-------------
| O |   |   |
-------------
```

est représentée par le 9-uplet `('X', 'O', 'X', '?', 'X', 'O', 'O', '?', '?')` et c'est à X de jouer.

Le choix du $n$-uplet plutôt que de la liste est fait car c'est une structure immuable. Or on va regarder beaucoup d'états du jeu et faire des retours en arrière: plutôt que de reconstruire l'état précédent, ce qui peut mener à des erreurs, on préfère avoir un nouvel état à chaque étape.

Pour vous aider à debugger les premières fonctions, je vous fournis une fonction d'affichage:

In [ ]:
def affiche(grille):
    print('-------------')
    for i in [0, 3, 6]:
        print('|'.join([' '+c+' ' for c in grille[i:3+i]]) + '|')
        print('-------------')

In [ ]:
affiche(('X', 'O', 'X', '?', 'X', 'O', 'O', '?', '?'))

### Exercice 1 : Détecter une grille pleine
Écrire une fonction `grille_pleine` qui prend une grille sous le format décrit précédemment, renvoie `True` si elle est pleine, `False` sinon.

In [ ]:
assert(grille_pleine(('X', 'O', 'X', '?', 'X', 'O', 'O', '?', '?')) == False)
assert(grille_pleine(('X', 'O', 'X', 'X', 'X', 'O', 'O', 'O', 'X')) == True)

### Exercice 2 : Joueur gagnant

Écrire une fonction `gagne` qui prend en argument une grille sous la forme décrite ci-dessus et une chaîne de longueur 1 représentant un joueur (`'X'` ou `'O'`) et qui renvoie `True` si le joueur en question possède trois symboles alignés dans la grille et `False` sinon.

In [ ]:
grille = ('X', 'O', 'X', 'X', 'X', 'O', 'O', 'O', 'X')
assert(gagne(grille, 'X') == True)
assert(gagne(grille, 'O') == False)

grille = ('X', 'O', 'X', '?', 'X', 'O', 'O', '?', '?')
assert(gagne(grille, 'X') == False)
assert(gagne(grille, 'O') == False)

### Exercice 3 : Coups possibles
Écrire une fonction `coups` qui prend en argument une grille supposée représenter un début de jeu (sans avoir à le vérifier) et renvoie la liste des indices des cases vides.

In [ ]:
grille = ('X', 'O', '?', '?', 'X', 'O', '?', 'O', 'X')
assert(set(coups(grille)) == set([2,3,6]))

grille = ('X', 'O', 'X', 'X', 'X', 'O', 'O', 'O', 'X')
assert(set(coups(grille)) == set([]))

### Exercice 4 : Coup joué
Écrire une fonction `jouer` qui prend en argument une grille, un joueur et un indice de case et renvoie une nouvelle grille dans laquelle le joueur a joué à l'indice fourni.

In [ ]:
grille = ('X', 'O', 'X', '?', 'X', 'O', 'O', '?', '?')
assert(jouer(grille, 'X', 3) == ('X', 'O', 'X', 'X', 'X', 'O', 'O', '?', '?'))

### Exercice 5 : Compter les sommets dans l'arbre des configurations
On veut maintenant compter les sommets dans l'arbre des configurations dont la racine correspond à une grille vide (donc dont toutes les cases contiennent `'?'`) et chaque arc à un coup joué par le joueur dont c'est le tour.

1. Écrire une fonction `adversaire` qui prend en argument le symbole d'un joueur et renvoie celui de son adversaire.

In [ ]:
assert(adversaire('O') == 'X')
assert(adversaire('X') == 'O')

2. Écrire une fonction `compter` qui prend en argument une grille et le prochain joueur à jouer et compte le nombre de sommets dans le sous-arbre de l'arbre des configurations de racine la grille fournie en argument (y compris la racine). Vous pourrez par exemple écrire une fonction récursive qui s'arrête sur un état final du jeu, et fait un appel récursif pour chaque coup possible. 

In [ ]:
grille = ('X', 'O', 'X', 'X', 'X', 'O', 'O', 'O', 'X')
assert(compter(grille, 'O') == 1)

grille = ('X', 'O', 'X', '?', 'X', 'O', 'O', '?', '?')
assert(compter(grille, 'X') == 12)

grille = ('X', 'O', 'X', '?', '?', '?', '?', '?', '?')
assert(compter(grille, 'O') == 1693)

3. Écrire une fonction `tout_compter` qui compte tous les sommets de l'arbre de configurations.

In [ ]:
assert(tout_compter() == 706978)

### Exercice 6 : Compter sans les doublons
On maintenant non plus compter tous les sommets de l'arbre des configurations, mais toutes les configurations différentes qui peuvent apparaître. Pour cela, on va utiliser une technique proche de la mémoïsation: on crée un tableau indicé par les configurations et dont toutes les cases contiennent `False`. Pour chaque configuration: soit on l'a déjà rencontrée et on s'arrête, soit elle est nouvelle et on continue en mettant la case correspondante à jour dans le tableau. Les configurations n'étant pas des entiers, on va utiliser une fonction de hachage sans collision pour leur associer des entiers.

1. À chaque grille, on associe une valeur hachée calculée de la façon suivante: à `'?'` on associe la valeur 0, à `'X'` la valeur 1 et à `'O'` la valeur 2. On considère ensuite la liste des cases de la grille dans leur ordre d'apparition comme un entier écrit en base 3. La valeur hachée de la grille est cet entier. Écrire une fonction `hash` qui prend en argument une grille et renvoie sa valeur hachée.

In [ ]:
grille = tuple('?')*9
assert(hash(grille) == 0)

grille = ('?', '?', '?', '?', '?', '?', '?', '?', 'X')
assert(hash(grille) == 1)

grille = ('?', '?', '?', '?', '?', '?', '?', '?', 'O')
assert(hash(grille) == 2)

grille = ('X', 'O', 'X', '?', 'X', 'O', 'O', '?', '?')
assert(hash(grille) == 11817)

grille = tuple('O')*9
assert(hash(grille) == 19682)

Clairement la fonction `hash` ne crée pas de collision entre deux grilles. On voit dans le dernier test qu'il  a un peu moins de 20000 valeurs hachées possibles.

2. Écrire une fonction `compter_sans_doublons` qui prend en argument une grille, un joueur et une liste de booléens dont on suppose que c'est une table de hachage (case `i` à `True` si et seulement si la valeur hachée `i` a déjà été vue) et renvoie le nombre de nouvelles configurations à partir de la grille fournie si c'est au joueur fourni en argument de jouer.

In [ ]:
grille = ('X', 'O', 'X', 'X', 'X', 'O', 'O', 'O', 'X')
assert(compter_sans_doublons(grille, 'O', [False]*20000) == 1)

grille = ('X', 'O', 'X', '?', 'X', 'O', 'O', '?', '?')
assert(compter_sans_doublons(grille, 'X', [False]*20000) == 11)

grille = ('X', 'O', 'X', '?', '?', '?', '?', '?', '?')
assert(compter_sans_doublons(grille, 'O', [False]*20000) == 257)

3. Écrire une fonction `tout_compter_sans_doublons` qui compte tous les sommets de l'arbre de configurations sans doublons.

In [ ]:
assert(tout_compter_sans_doublons() == 5691)